## 수치 미분을 이용한 심층 신경망 학습  
### 이번에는 역전파와 그렇지 않은 수치미분을 비교해 봅니다. 지금은 역전파가 아닌 수치 미분입니다.  
### (비효율적인 프로그램)
time을 이용해서 학습을 하는데 얼마나 시간이 걸리는지 측정해봅니다.

In [15]:
import time
import numpy as np

## 유틸리티 함수  
수치미분할때 입실론으로 나누고 입실론으로 더해서 로스펑션을 evaluate하는 부분이 들어가니 입실론을 정의해 줍니다.  
매번 길게 쓰기는 힘드니 transpose는 _ t로 쓰겠습니다.  
matmul도 _ m으로 쓰겠습니다. 
sigmoid 도 구현을 해 놓습니다.


In [16]:
epsilon = 0.0001

def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mean_squared_error(h, y):
    return 1 / 2 * np.mean(np.square(h - y))

## 뉴런 구현  
w b a 를 받으면 저장합니다.  
모델 파라메터들을 입력받으며, 각각의 그라디언트를 저장하기 위한 변수도 따로 만들어 줍니다.  
dW라고 해서 d로 미분했다고 합시다.
dw는 W와 똑같은 numpy object의 0으로 구성된 matrix가 생성됩니다.  
bias도 마찬가지로 만들어 줍니다. 그러면 크기를 따로 입력하지 않아도 쉽게  만들 수 있습니다.  

뉴런은 콜을했을때 activation function을 입력 받습니다. 그러면 W와 X를 곱할껀데 그전에 transpose를 넣어 줍니다. 

In [17]:
class Neuron:
    def __init__(self, W, b, a):
        #Model Parameter
        self.W=W
        self.b=b
        self.a=a
        
        #Gradients
        self.dW=np.zeros_like(self.W)
        self.db=np.zeros_like(self.b)

    def __call__(self, x):
        return self.a(_m(_t(self.W),x)+self.b) #activation((W^T)x+b)

## 심층신경망 구현
init_var은 첫번째 출력이 random normal input output이다. 0이 평균이고 0.01이 standard variation입니다.  
i by o의 메트릭스로 리턴하구요.   
출력크기에 맞는 바이어스를 제로로 출력하는 메트릭스도 같이 리턴을 해줍니다.  
그래서 w와 b를 initialize해줍니다.

입력을 살펴보죠.  
hidden_depth : 히든레이어가 몇개 들어갈 것인지.  
num_neuron : 뉴런의 개수 (히든레이어 하나당)  
input : input 레이어의 뉴런 개수
output : output 레이어의 뉴런 개수
activation finction은 기본적으로 sigmoid로 넣어줍니다.  
relu라던지 다른 테스트삼아 구현해 넣어보시는 것도 좋습니다.  

먼저 첫번째 히든레이어를 구현해 봅니다.  
뉴런은 W,b,Activation을 입력으로 받는것을 기억합니다.  

first가 아닌 리든레이어들은 여러개가 들어가니 for문을 사용해줍니다. 첫번째에 히든레이어를 만들어 줬으니 hidden_depth-1을 해줍니다. 

마지막은 activation을 softmax나 linear로 바꾸면 clssification등으로 구현할 수 있습니다. 지금은 그저 구현목적이니 이대로 냅두겠습니다.

# calc_gradient


In [33]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))

        self.sequence = list()
        # First hidden layer
        W,b=init_var(num_input,num_neuron)
        self.sequence.append(Neuron(W,b,activation))
        
        # Hidden layers
        for _ in range(hidden_depth -1):
            W,b=init_var(num_neuron,num_neuron)
            self.sequence.append(Neuron(W,b,activation))

        # Output layer
        W,b=init_var(num_neuron,num_output)
        self.sequence.append(Neuron(W,b,activation))

        #layer을 콜하는것은 뉴런을 콜하는 것입니다.
        #그럼 한단계의 뉴런을 실행합니다. 위에 있는 리턴대로요
    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, x, y, loss_func):
        def get_new_sequence(layer_index,new_neuron):
            new_sequence=list()
            for i, layer in enumerate(self.sequence):
                if i == layer_index:
                    new_sequence.append(new_neuron)
                else:
                    new_sequence.append(layer)
            return new_sequence
        
        def eval_sequence(x,sequence):
            for layer in sequence:
                x=layer(x)
            return x
        
        loss=loss_func(self(x),y)
        
        #self(x) = 추정한 것
        #시퀀스를 돌면서 레이어를 뽑고, 웨이트에 하나하나 컬럼들이 나오게 된다.
        
        for layer_id, layer in enumerate(self.sequence):
            for w_i,w in enumerate(layer.W):
                for w_j,ww in enumerate(w):
                    W=np.copy(layer.W)
                    W[w_i][w_j]=ww+epsilon
                    new_neuron=Neuron(W,layer.b,layer.a)
                    new_seq=get_new_sequence(layer_id,new_neuron)
                    h=eval_sequence(x,new_seq)
                    
                    num_grad=(loss_func(h,y)-loss)/epsilon #(f(x+eps)-f(x))/epsilon
                    layer.dW[w_i][w_j]=num_grad
                for b_i,bb in enumerate(layer.b):
                    b=np.copy(layer.b)
                    b[b_i]=bb+epsilon
                    new_neuron=Neuron(W,layer.b,layer.a)
                    new_seq=get_new_sequence(layer_id,new_neuron)
                    h=eval_sequence(x,new_seq)
                    
                    num_grad=(loss_func(h,y)-loss)/epsilon #(f(x+eps)-f(x))/epsilon
                    layer.db[b_i]=num_grad
            return loss # 매번 첫번째 loss를 리턴한다.

## 경사하강 학습법  
네트워크 입력, 학습데이터 입출력, 어떤로스쓸건지, 학습률을 0.01로 세팅  
 입출력(데이터셋)과 로스받아서 그라디언트 계산해 loss로 받아준다.  
 그라디언트가 계산된 레이어들을 순환한다. 

In [34]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = network.calc_gradient(x, y, loss_obj)
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

## 동작 테스트

In [35]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

dnn = DNN(hidden_depth=5, num_neuron=32, num_input=10, num_output=2, activation=sigmoid)

t = time.time()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, mean_squared_error, 0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 0.07886313075266332
Epoch 1: Test loss 0.07886313075266332
Epoch 2: Test loss 0.07886313075266332
Epoch 3: Test loss 0.07886313075266332
Epoch 4: Test loss 0.07886313075266332
Epoch 5: Test loss 0.07886313075266332
Epoch 6: Test loss 0.07886313075266332
Epoch 7: Test loss 0.07886313075266332
Epoch 8: Test loss 0.07886313075266332
Epoch 9: Test loss 0.07886313075266332
Epoch 10: Test loss 0.07886313075266332
Epoch 11: Test loss 0.07886313075266332
Epoch 12: Test loss 0.07886313075266332
Epoch 13: Test loss 0.07886313075266332
Epoch 14: Test loss 0.07886313075266332
Epoch 15: Test loss 0.07886313075266332
Epoch 16: Test loss 0.07886313075266332
Epoch 17: Test loss 0.07886313075266332
Epoch 18: Test loss 0.07886313075266332
Epoch 19: Test loss 0.07886313075266332
Epoch 20: Test loss 0.07886313075266332
Epoch 21: Test loss 0.07886313075266332
Epoch 22: Test loss 0.07886313075266332
Epoch 23: Test loss 0.07886313075266332
Epoch 24: Test loss 0.07886313075266332
Epoch 25: 